### For loops to pull L2A, L2B, L4A as needed

In [1]:
!pip install geopandas
!pip install contextily

  Using cached contextily-1.6.2-py3-none-any.whl.metadata (2.9 kB)
Using cached contextily-1.6.2-py3-none-any.whl (17 kB)


In [2]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
   
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
   
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

def wait_for_job(job_id: str) -> str:
    return job_status_for(job_id)

In [3]:
#pip install geopandas
#pip install shapely
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd
import backoff
import shapely
import glob
import os

In [4]:
import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [5]:
# # Define the folder path containing the tiles
# #folder_path = "/projects/my-public-bucket/AOIs/GHANA/"
# folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/"

# # Create a list to store the GeoPackage file paths
# json_files = []

# # Use glob to find all .gpkg files in the folder
# #pattern = os.path.join(folder_path, '*.geojson')
# pattern = os.path.join(folder_path, 'Mali_South.gpkg')
# json_files = glob.glob(pattern)

# # Print the list of GeoPackage file paths
# #for file_path in gpkg_files:
#  #   print(file_path)

# #json_files = [x.replace('/projects/my-public-bucket/AOIs/GHANA/',
# #                        'https://maap-ops-workspace.s3.amazonaws.com/shared/leitoldv/AOIs/GHANA/') for x in json_files]
# json_files = [x.replace('/projects/my-public-bucket/Data/NASA_CMS_2023/WA_15_countries/',
#                         'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/WA_15_countries/') for x in json_files]

# print(len(json_files))
# print(json_files)
# AOIs = json_files
# AOIs[0]

In [6]:
# # OPTIONAL # visually verify the AOI
# aoi = AOIs[0]

# try:
#     import geopandas as gpd
#     import contextily as ctx
# except:
#     print(
#         "If you wish to visually verify your AOI, "
#         "you must install the `geopandas` and `contextily` packages."
#     )
# else:
#     aoi_gdf = gpd.read_file(aoi)
#     aoi_epsg4326 = aoi_gdf.to_crs(epsg=4326)
#     ax = aoi_epsg4326.plot(figsize=(10, 5), alpha=0.3, edgecolor="red")
#     ctx.add_basemap(ax, crs=4326)

In [7]:
# aoi_name = os.path.basename(AOIs[0]).split('.')[0]
# aoi_name

In [8]:
# folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/"
# json_files = []
# pattern = os.path.join(folder_path, '*.gpkg')
# json_files = glob.glob(pattern)
# json_files = [ x for x in json_files if "L2A" not in x ]
# json_files = [x.replace('/projects/my-public-bucket/Data/NASA_CMS_2023/MONGOLIA/GRID/',
#                         'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MONGOLIA/GRID/') for x in json_files]
# print(len(json_files))
# print(json_files)
# AOIs = json_files
# AOIs[0]

folder_path = "/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/GEDI_Strata/"
json_files = []
pattern = os.path.join(folder_path, 'Id_1*.gpkg')
json_files = glob.glob(pattern)
json_files = [ x for x in json_files if "L2A" not in x ]
json_files = [x.replace('/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/GEDI_Strata/',
                        'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/GEDI_Strata/') for x in json_files]
print(len(json_files))
print(json_files)
AOIs = json_files
AOIs[0]

1
['https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/GEDI_Strata/Id_1.gpkg']


'https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/SUDAN/GEDI_Strata/Id_1.gpkg'

In [9]:
#Set up run to pull products for all GEDI products simultaneously #To run for only 1 or 2 products, change "PRODUCTS"

PRODUCTS = ["L4A"] # ,"L2A","L2B"]  #To run for only 1 or 2 products, change "PRODUCTS"

AOIs = json_files
#AOIs = [AOIs[2]]

for each_aoi in AOIs: 
    if "L4A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L4A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L4A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           columns="shot_number,lat_lowestmode,lon_lowestmode,elev_lowestmode,agbd,agbd_se,agbd_t,agbd_t_se,sensitivity,geolocation/sensitivity_a2",
           query="l2_quality_flag == 1 and l4_quality_flag == 1 and sensitivity > 0.95", #  and geolocation/sensitivity_a2 > 0.95",
           limit = 100000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_SUD_L4A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32vcpu-64gb",
            username="nehajo88",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result

    if "L2B" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2B subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2B",
           lat="geolocation/lat_lowestmode",
           lon="geolocation/lon_lowestmode",
           beams="all",
           columns="shot_number,geolocation/lon_lowestmode,geolocation/lat_lowestmode,rh100,l2b_quality_flag,sensitivity,cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
           query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
           limit = 100000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_SUD_L2B.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32vcpu-64gb",
            username="nehajo88",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result
        
        
    if "L2A" in PRODUCTS: 
        aoi_name = os.path.basename(each_aoi).split('.')[0]
        print(f"Thanks, I'll run the GEDI L2A subsetter for {aoi_name}!!")
        #aoi_url = each_aoi
        inputs = dict(
           aoi=each_aoi,
           doi="L2A",
           lat="lat_lowestmode",
           lon="lon_lowestmode",
           beams="all",
           #columns="shot_number,lon_lowestmode,lat_lowestmode,rh25,rh50,rh75,rh90,rh98,geolocation/sensitivity_a2",
           columns="shot_number,lon_lowestmode,lat_lowestmode,rh10,rh20,rh30,rh40,rh50,rh60,rh70,rh80,rh90,rh98,quality_flag,rx_processing_a2/rx_algrunflag,rx_processing_a2/zcross,rx_processing_a2/toploc,geolocation/sensitivity_a2,degrade_flag,land_cover_data/pft_class,land_cover_data/region_class,surface_flag,rx_assess/rx_maxamp,rx_assess/sd_corrected,geolocation/stale_return_flag,land_cover_data/leaf_off_flag,land_cover_data/landsat_water_persistence,land_cover_data/urban_proportion",
           #columns="all",
           query="quality_flag == 1",
           limit = 100000,
           #temporal="-",
           output=f"{os.path.basename(each_aoi).split('.')[0]}_SUD_L2A.gpkg"
        )
        result = maap.submitJob(
            identifier="gedi-subset",
            algo_id="gedi-subset",
            version="0.9.0",
            queue="maap-dps-worker-32vcpu-64gb",
            username="nehajo88",
            **inputs
        )
        inputs
        job_id = result.id
        job_id or result
        

Thanks, I'll run the GEDI L4A subsetter for Id_1!!


In [10]:
import subprocess

root_dir = "/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2024/10/"  # Can now set name
out_dir = "/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/"

for dirName, subdirList, fileList in os.walk(root_dir):
#    print("Found directory: %s" % dirName)
    for fname in fileList:
        if fname.endswith("_SUD_L2A.gpkg") or fname.endswith("_SUD_L4A.gpkg"):
#        if fname.endswith("NPAs_L4A.gpkg"):
            source_path = os.path.join(dirName, fname)
            print(source_path)
            target_path = os.path.join(out_dir, fname)
            print(target_path)
            if not os.path.exists(target_path):  # Check if file doesn’t exist in target directory
                print(f"Moving {fname}...")
                subprocess.call(['mv', source_path, target_path])
            else:
                print(f"{fname} already exists in the target directory. Skipping...")

/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2024/10/30/21/53/34/315320/Id_1_SUD_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_1_SUD_L4A.gpkg
Moving Id_1_SUD_L4A.gpkg...


mv: failed to close '/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_1_SUD_L4A.gpkg': Input/output error


/projects/my-private-bucket/dps_output/gedi-subset/0.9.0/gedi-subset/2024/10/31/16/15/44/334758/Id_1_SUD_L4A.gpkg
/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_1_SUD_L4A.gpkg
Id_1_SUD_L4A.gpkg already exists in the target directory. Skipping...


In [11]:
import rasterio
import os
import argparse
from rasterio.io import MemoryFile
from rasterio.rio import options
from rasterio.shutil import copy, delete
from rasterio.vrt import WarpedVRT
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
from rasterio.transform import from_origin
from rasterio.crs import CRS
import pandas as pd
import geopandas as gpd
import numpy as np

In [27]:
L2A = gpd.read_file(os.path.join(FOLDER,fname))
L2A.drop('geometry',axis=1).to_csv(r'/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_3_SUD_L4A.csv') 

In [28]:
L2A = gpd.read_file('/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_4_SUD_L4A.gpkg')
L2A.drop('geometry',axis=1).to_csv(r'/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_4_SUD_L4A.csv') 

In [4]:
L2A = gpd.read_file('/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_2_SUD_L4A.gpkg')
L2A.drop('geometry',axis=1).to_csv(r'/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_2_SUD_L4A.csv') 

In [12]:
L2A = gpd.read_file('/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_1_SUD_L4A.gpkg')
L2A.drop('geometry',axis=1).to_csv(r'/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_1_SUD_L4A.csv') 

ERROR:fiona._env:database disk image is malformed
ERROR:fiona._env:sqlite3_prepare_v2(SELECT COUNT(*) FROM sqlite_master WHERE name IN ('gpkg_metadata', 'gpkg_metadata_reference') AND type IN ('table', 'view')) failed: database disk image is malformed


DriverError: sqlite3_prepare_v2(SELECT COUNT(*) FROM sqlite_master WHERE name IN ('gpkg_metadata', 'gpkg_metadata_reference') AND type IN ('table', 'view')) failed: database disk image is malformed

In [ ]:
import rasterio
import os
import argparse
from rasterio.io import MemoryFile
from rasterio.rio import options
from rasterio.shutil import copy, delete
from rasterio.vrt import WarpedVRT
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles
from rasterio.transform import from_origin
from rasterio.crs import CRS
import pandas as pd
import geopandas as gpd
import numpy as np

FOLDER = "/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/"

for dirName, subdirList, fileList in os.walk(FOLDER):
    for fname in fileList:
        if fname.endswith("_SUD_L4A.csv"):
            OUTFILE = os.path.join(FOLDER,fname.split('.')[0] + '.tif')
            print(OUTFILE)
            if os.path.exists(OUTFILE): 
                os.remove(OUTFILE)
            L2A = pd.read_csv(os.path.join(FOLDER,fname))
            
            # L2A = gpd.read_file(os.path.join(FOLDER,fname))
            # L2A = L2A.drop('geometry',axis=1)
            
            x = L2A["lon_lowestmode"]
            y = L2A["lat_lowestmode"]
            # z = L2A.rh98
            z = L2A.agbd
            
            pixel_width = 0.0025 #units in degrees
            pixel_height = 0.0025 #units in degrees
            
            nx = int(np.round((np.max(x) - np.min(x))/pixel_width)) # nx is the size of the output raster in x direction
            ny = int(np.round((np.max(y) - np.min(y))/pixel_width)) # ny is the size of the output raster in x direction
            
            zi, yi, xi = np.histogram2d(y, x, bins=(ny,nx), weights=z)
            counts, _, _ = np.histogram2d(y, x, bins=(ny,nx))
                        
            zi = np.divide(zi, counts, out=np.zeros_like(zi), where=counts != 0)
            
            z = np.array(zi.data[::-1])
            z[np.isnan(z)] = 0

            pixel_width = xi[1] - xi[0] # approximation needed again so as to match number bins and pixels
            pixel_height = yi[1] - yi[0] # approximation needed again so as to match number bins and pixels
            x_min = xi[0]# - pixel_width / 2
            y_max = yi[np.size(yi)-1]# + pixel_height / 2
            
            transform = from_origin(x_min, y_max, pixel_width, pixel_height)
            
            crs = CRS.from_epsg(4326)
            
            kwargs = {
                'driver': 'GTiff',
                'dtype': z.dtype,
                'count': 1,  # Number of bands
                'height': z.shape[0],
                'width': z.shape[1],
                'crs': crs,
                'transform': transform,
                'predictor': 2
            }
            
            with MemoryFile() as memfile:
                with memfile.open(**kwargs) as mem:
                    # Populate the input file with numpy array
                    mem.write(z.reshape(1, z.shape[0], z.shape[1]))
            
                    dst_profile = cog_profiles.get("deflate")
                    cog_translate(
                        mem,
                        OUTFILE,
                        dst_profile,
                        in_memory=False
                    )
            
            print("GeoTIFF has been created successfully with GDAL.")

/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_2_SUD_L4A.tif


/tmp/ipykernel_168/1644180605.py:75: RasterioDeprecationWarning: Source dataset should be opened in read-only mode. Use of datasets opened in modes other than 'r' will be disallowed in a future version.
  cog_translate(
Reading input: <open DatasetWriter name='/vsimem/eaeedad1-4c04-4721-a7fe-b5e828e5cc95/eaeedad1-4c04-4721-a7fe-b5e828e5cc95.tif' mode='w+'>

Adding overviews...
Updating dataset tags...
Writing output to: /projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_2_SUD_L4A.tif


GeoTIFF has been created successfully with GDAL.
/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_3_SUD_L4A.tif


/tmp/ipykernel_168/1644180605.py:75: RasterioDeprecationWarning: Source dataset should be opened in read-only mode. Use of datasets opened in modes other than 'r' will be disallowed in a future version.
  cog_translate(
Reading input: <open DatasetWriter name='/vsimem/0418b01b-aa20-4004-aa41-6c78032281bb/0418b01b-aa20-4004-aa41-6c78032281bb.tif' mode='w+'>

Adding overviews...
Updating dataset tags...
Writing output to: /projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_3_SUD_L4A.tif


GeoTIFF has been created successfully with GDAL.
/projects/my-public-bucket/Data/NASA_CMS_2023/SUDAN/Id_4_SUD_L4A.tif
